# Import libraries and define directory
Loading necessary libraries and defininf the data file directory

In [1]:
# Import Libraries
from pathlib import Path
import os
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objs as go
import math
import plotly.offline as pyo

# Set directory to where the data file is located
files_directory = Path().resolve()
os.chdir(files_directory)


## Load data
Load the data from the same directory of this code and inspect.

In [2]:
# Load data file
data_filename ='data36.xlsx'
data = pd.read_excel(data_filename)
print(data.head())  # Display first few rows of data

                ID          X           Y  PROF     HFL      HFM       HFP  \
0  SPCP209-17 1.0   392153.16  2015541.05   1.0  130.08  2686.25  10695.21   
1  SPCP209-12 0.2   392153.16  2015541.05   0.2  133.07  2895.64  10542.95   
2  SPCP209-12 1.0   392153.16  2015541.05   1.0  115.90  2324.42   9782.14   
3  SPCP209-10 0.2   392097.16  2015541.05   0.2  122.81  2624.60   9247.28   
4  SPCP209-11 0.2   392125.16  2015541.05   0.2  115.76  2454.35   8763.25   

   BTEX  HAP  
0     0    0  
1     0    0  
2     0    0  
3     0    0  
4     0    0  


# Define relevant variables
Assign the name of columns for UTM X & Y coordinates, depth, pollutant concentration and its threshold value.

In [3]:
# Define key variables
x_column = 'X'
y_column = 'Y'
z_column = 'PROF'
pollutant_column = 'HFM'
MPL = 1200
units = 'mg/kg'

x = data[x_column].values  # UTM X coordinate
y = data[y_column].values  # UTM Y coordinate
z = data[z_column].values * -1  # Depth values (inverted)
pollutant_concentration = data[pollutant_column].values  # Pollutant concentration

points_x = x
points_y = y
points_z = z
points_id = data['ID'].values #ID column

# Generate 3D grid
Generate a 3D grid to interpolate the data over the defined spatial region.

In [4]:
# Create a 3D grid
grid_x, grid_y, grid_z = np.mgrid[
    min(x):max(x):1,
    min(y):max(y):1,
    min(z):0+0.2:0.2
]

## Interpolate data on the 3D grid
Interpolate the pollutant data onto the 3D grid.

In [5]:
# Interpolation method (linear or nearest)
method = 'linear'
grid_pollutant = griddata((x, y, z), pollutant_concentration, (grid_x, grid_y, grid_z), method=method)

# 3D visualization of pollutant distribution
Create a 3D visualization of the pollutant distribution using Plotly.

In [6]:
#Plot titles
x_title = 'E'
y_title = 'N'
z_title = 'Z'
color_title = 'Concentración'
title = 'HFM in soil'

In [7]:
# 3D Visualization
max_plot = math.ceil(pollutant_concentration.max() / MPL) * MPL

# Grid
fig = go.Figure(data=go.Volume(
    x=grid_x.flatten(),
    y=grid_y.flatten(),
    z=grid_z.flatten(),
    value=grid_pollutant.flatten(),
    isomin=0,
    isomax=max_plot,
    opacity=0.2,
    surface_count=int(max_plot/MPL)+1,
    colorbar=dict(title=f"{color_title} ({units})"),
    colorscale="bluered"
))

#Sampling points
fig.add_trace(go.Scatter3d(
    x=points_x,
    y=points_y,
    z=points_z,
    mode='markers',
    marker=dict(
        size=2,
        color='black',
        opacity=0.8
    ),
    showlegend=False,
    text=points_id,
    hoverinfo="text"
))

# Set aspect ratio
x_range = max(x) - min(x)
y_range = max(y) - min(y)
ratio_y = y_range / x_range

# Configure figure layout
fig.update_layout(
    scene=dict(
        xaxis_title=x_title,
        yaxis_title=y_title,
        zaxis_title=z_title,
    ),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=ratio_y, z=0.25),
    title=title
)

_ = None

# Estimation of polluted volume
Estimate the volume where the pollutant concentration exceeds the Maximum Permissible Limit (MPL)

In [8]:
# Create a mask for cells with concentrations above MPL
mask = grid_pollutant > MPL

# Determine grid resolution
delta_x = (max(x) - min(x)) / grid_x.shape[0]
delta_y = (max(y) - min(y)) / grid_y.shape[1]
delta_z = (max(z) - min(z)) / grid_z.shape[2]

# Calculate cell volume
volume_cell = delta_x * delta_y * delta_z

# Count cells exceeding MPL and calculate total polluted volume
num_cells_above = np.sum(mask)
volume_total_above = num_cells_above * volume_cell

print(f"The soil volume with pollutant concentrations above MPL is: {volume_total_above:.2f} m^3")

The soil volume with pollutant concentrations above MPL is: 869.79 m^3


# Determine maximum pollution depth
Identify the maximum depth where pollutant concentration exceeds MPL.

In [9]:
# Maximum pollution depth
depths_above = grid_z[mask]
maximum_depth = np.min(depths_above)

print(f"Maximum depth with pollutant concentration above MPL: {maximum_depth:.2f} m")

Maximum depth with pollutant concentration above MPL: -2.00 m


# 3D visualization of polluted volume
Create a filtered 3D visualization showing only the volume exceeding MPL.

In [10]:
# 3D Visualization
max_plot = math.ceil(pollutant_concentration.max() / MPL) * MPL

fig_polluted = go.Figure(data=go.Volume(
    x=grid_x.flatten(),
    y=grid_y.flatten(),
    z=grid_z.flatten(),
    value=grid_pollutant.flatten(),
    isomin=MPL,
    isomax=max_plot,
    opacity=0.25,
    surface_count=int((max_plot-MPL)/MPL)+1,
    colorbar=dict(title=f"{color_title} ({units})"),
    colorscale="Sunset"
))

#Sampling points
fig_polluted.add_trace(go.Scatter3d(
    x=points_x,
    y=points_y,
    z=points_z,
    mode='markers',
    marker=dict(
        size=2,
        color='black',
        opacity=0.8
    ),
    showlegend=False,
    text=points_id,
    hoverinfo="text"
))

# Set aspect ratio
x_range = max(x) - min(x)
y_range = max(y) - min(y)
ratio_y = y_range / x_range

# Configure figure layout
fig_polluted.update_layout(
    scene=dict(
        xaxis_title=x_title,
        yaxis_title=y_title,
        zaxis_title=z_title,
    ),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=ratio_y, z=0.25),
    title=str(title)+'(>'+str(MPL)+')'
)
#fig_polluted.show()
_ = None

# Export visualization as HTML
Save the visualizations as HTML files.

In [11]:
# Save visualizations as HTML
pyo.plot(fig, filename='volumetric_visualization.html', auto_open=False)
pyo.plot(fig_polluted, filename='polluted_visualization.html', auto_open=False)

'polluted_visualization.html'